Transpose / Pivot table Pandas

In [ ]:
import pandas as pd

df.set_index('Material Name', inplace=True)

trans = df.transpose()

trans.reset_index(inplace=True)

trans = trans[cols]

# One way to do it, for now it can not remain same order easily.
trans.melt(id_vars=["index"], ignore_index = False).to_excel('../output/hope.xlsx')

# Another way but preverse order.
result= trans.set_index(["index"]).stack()
# Or
result = df.pivot(index=["some cols"], columns="cols contain values to set name of columns", values="cols that want to get values from")

Convert to datetime

In [ ]:
df['columns name'] = pd.to_datetime(df['columns name']], format="%d/%m/%Y", yearfirst=True)

Dataframe to SQL

In [ ]:
from sqlalchemy import create_engine

df = pd.read_excel('Byun/take_in.xlsx', header=10)

sql_engine = create_engine(
        'mysql+pymysql://admin:password@ip/scheme', pool_recycle=3600
    )
take_in.to_sql('table', sql_engine,
              if_exists='replace')